In [10]:
import streamlit as st
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import folium
import zipfile
import os

st.set_page_config(page_title="ENEM Analysis",
                   page_icon=":bar_chart:",
                   layout="wide")

In [11]:
# Configurações do Pandas
pd.options.display.max_columns = 26

In [12]:
def build_dataframe():
    # Arrays e dicionários úteis
    columns = ["NU_ANO","TP_NACIONALIDADE","TP_SEXO","TP_FAIXA_ETARIA","TP_COR_RACA","TP_ESTADO_CIVIL",
            "TP_ST_CONCLUSAO","TP_ESCOLA","IN_TREINEIRO","NO_MUNICIPIO_ESC","SG_UF_PROVA","TP_PRESENCA_CN",
            "TP_PRESENCA_CH","TP_PRESENCA_LC","TP_PRESENCA_MT","NU_NOTA_CN","NU_NOTA_CH","NU_NOTA_LC",
            "NU_NOTA_MT","TP_STATUS_REDACAO","NU_NOTA_COMP1","NU_NOTA_COMP2","NU_NOTA_COMP3","NU_NOTA_COMP4",
            "NU_NOTA_COMP5","NU_NOTA_REDACAO"]

    renamed_columns = ["Ano","Nacionalidade","Sexo","Idade","Cor","Estado_civil","Situacao_EM","Tipo_Escola",
                    "Treineiro","Municipio","Estado","Presenca_CN","Presenca_CH","Presenca_LC","Presenca_MT",
                    "Ciencias_Natureza","Ciencias_Humanas","Linguagens_Codigos","Matematica","Status_Redacao",
                    "Ortografia","Desenvolvimento","Informacoes","Organizacao","Proposta","Redacao"]
    
    zip_data = {
    "Zip Files":[],
    "Folder Data":  os.listdir("information")
    }

    for zippedArch in zip_data["Folder Data"]:
        with zipfile.ZipFile(f"information\{zippedArch}") as zippedData:
            for file in zippedData.namelist():
                if "microdados" in file.lower() and file.endswith(".csv"):
                    zip_data["Zip Files"].append(file)
                    
    # Lambda pra ordenar os arquivos do mais antigo para o mais novo 2015 - 2022
    zip_data["Zip Files"].sort(key = lambda x: x[22:-4])
    zip_data["Folder Data"].sort(key = lambda x: x[16:-4])
    
    # Pegando os ultimos 3 anos
    zip_data["Folder Data"] = zip_data["Folder Data"][-1:]
    zip_data["Zip Files"] = zip_data["Zip Files"][-1:]

    # Criando o megadataframe filtrado
    enem_collection = []
    for i in range(len(zip_data["Zip Files"])):
        with zipfile.ZipFile(f"information\{zip_data['Folder Data'][i]}") as microdata:
            with microdata.open(zip_data["Zip Files"][i]) as csv:
                
                data = pd.read_csv(csv,encoding="ISO-8859-1",sep=";",usecols=columns)
                # Renomeando as colunas
                data.columns = renamed_columns
                # Removendo as linhas onde não temos notas
                data.dropna(subset=["Ciencias_Natureza",'Ciencias_Humanas',"Ciencias_Humanas","Matematica","Redacao"],inplace=True)
                data.reset_index(inplace=True,drop=True)
                # Adicionando no array pra criar o Dataframe completo posteriormente
                enem_collection.append(data)
                
    return pd.concat(enem_collection)
    

In [13]:
enem = build_dataframe()


In [14]:
# Calculando a Nota Total de cada pessoa
enem['Nota Total'] = enem[['Ciencias_Natureza','Ciencias_Humanas','Matematica','Linguagens_Codigos','Redacao']].sum(axis=1)/5

In [15]:
enem.head()

,Ano,Nacionalidade,Sexo,Idade,Cor,Estado_civil,Situacao_EM,Tipo_Escola,Treineiro,Municipio,Estado,Presenca_CN,Presenca_CH,...,Presenca_MT,Ciencias_Natureza,Ciencias_Humanas,Linguagens_Codigos,Matematica,Status_Redacao,Ortografia,Desenvolvimento,Informacoes,Organizacao,Proposta,Redacao,Nota Total
0,2022,5,F,1,2,1,1,1,0,NaN,BA,1,1,...,1,421.1,546.0,498.8,565.3,1.0,140.0,200.0,120.0,140.0,160.0,760.0,558.24
1,2022,6,M,1,3,1,1,1,0,NaN,ES,1,1,...,1,490.7,388.6,357.8,416.0,1.0,120.0,40.0,40.0,100.0,20.0,320.0,394.62
2,2022,6,F,1,2,1,1,1,0,NaN,RJ,1,1,...,1,398.1,427.3,400.2,404.9,1.0,100.0,120.0,80.0,100.0,40.0,440.0,414.10
3,2022,4,F,1,3,1,1,1,0,NaN,PE,1,1,...,1,467.5,461.0,466.7,435.3,1.0,80.0,80.0,80.0,80.0,40.0,360.0,438.10
4,2022,2,F,1,1,1,2,3,0,NaN,SE,1,1,...,1,458.7,539.8,488.2,456.8,1.0,160.0,200.0,180.0,200.0,200.0,940.0,576.70


In [16]:
def main():
    # Configuração inicial do Streamlit
    st.title("Mapa do Brasil usando Folium")
    st.sidebar.header("Configurações do Mapa")

    # Configurações do mapa
    zoom_level = st.sidebar.slider("Selecione o nível de zoom", 5, 15, 5)
    map_width = st.sidebar.slider("Selecione a largura do mapa", 500, 1500, 1000)

    # Criar o mapa do Brasil usando Folium
    mapa_brasil = folium.Map(location=[-15.788497, -47.879873], zoom_start=zoom_level, control_scale=True)

    # Adicionar limites do Brasil
    folium.GeoJson(
        "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson",
        name="geojson"
    ).add_to(mapa_brasil)

    # Renderizar o mapa no Streamlit
    folium_static(mapa_brasil, width=map_width)

def folium_static(fig, width=700, height=500):
    """Função para renderizar o mapa Folium no Streamlit."""
    fig.add_child(folium.LatLngPopup())  # Adicionar popup de coordenadas
    fig = fig._repr_html_().replace("width: 100%;", f"width: {width}px; height: {height}px;")
    st.write(folium.Html(fig, width=width, height=height, script=True))
